In [1]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice

from annsa.template_sampling import *

In [2]:
import tensorflow.contrib.eager as tfe

In [3]:
tf.enable_eager_execution()

#### Import model, training function 

In [4]:
from annsa.model_classes import (cnn1d_model_features,
                                 CNN1D,
                                 generate_random_cnn1d_architecture,
                                 save_model,
                                 train_earlystop)

## Load testing dataset 

In [5]:
dataset = np.load('../dataset_generation/testing_dataset_200keV_100.npy')

In [6]:
all_spectra = np.add(dataset.item()['sources'], dataset.item()['backgrounds'])
all_keys = dataset.item()['keys']

mlb=LabelBinarizer()

all_keys_binarized = mlb.fit_transform(all_keys)

## Train network

### Define hyperparameters

In [7]:
number_hyperparameters_to_search = 256
earlystop_errors_test = []
model_id='CNN-kfoldseasy'

In [8]:
def generate_random_cnn1d_architecture(cnn_filters_choices,
                                       cnn_kernel_choices,
                                       pool_size_choices):
    """
    Generates a random 1d convolutional neural network based on a set of
    predefined architectures.

    Returns
    -------
    model_features : class
        Class that describes the structure of a 1D convolution neural network.

    """

    cnn_filters = choice(cnn_filters_choices)
    cnn_kernel_choice = choice(cnn_kernel_choices)
    pool_size_choice = choice(pool_size_choices)

    cnn_kernel = cnn_kernel_choice*(len(cnn_filters))
    cnn_strides = (1,)*(len(cnn_filters))
    pool_size = pool_size_choice*(len(cnn_filters))
    pool_strides = (2,)*(len(cnn_filters))

    number_layers = np.random.randint(1, 4)
    dense_nodes = (10**np.random.uniform(1,
                                         np.log10(1024/(2**len(
                                             cnn_filters))),
                                         number_layers)).astype('int')
    dense_nodes = np.sort(dense_nodes)
    dense_nodes = np.flipud(dense_nodes)

    model_features = cnn1d_model_features(
            trainable=None,
            learining_rate=None,
            batch_size=None,
            output_size=None,
            scaler=None,
            activation_function=None,
            output_function=None,
            Pooling=None,
            l2_regularization_scale=None,
            dropout_probability=None,
            cnn_filters=cnn_filters,
            cnn_kernel=cnn_kernel,
            cnn_strides=cnn_strides,
            pool_size=pool_size,
            pool_strides=pool_strides,
            dense_nodes=dense_nodes
            )

    return model_features

def make_model():
    '''
    Makes a random model given some parameters.

    '''
    cnn_filters_choices = ((4, 8),
                       (8, 16),
                       (16, 32),
                       (4,),
                       (8,),
                       (16,),
                       (32,),
                       (4, 8, 16),
                       (8, 16, 32),
                      )

    
    cnn_kernel_choices = ((2,), (4,), (8,), (16,))
    pool_size_choices = ((2,), (4,), (8,), (16,))

    
    model_features = generate_random_cnn1d_architecture(
        cnn_filters_choices= cnn_filters_choices,
        cnn_kernel_choices=cnn_kernel_choices,
        pool_size_choices=pool_size_choices,
    )
    model_features.trainable = True
    model_features.learining_rate = 10**np.random.uniform(-4,-1)
    model_features.batch_size = 2**np.random.randint(4,6)
    model_features.output_size = all_keys_binarized.shape[1]
    model_features.scaler = choice([make_pipeline(FunctionTransformer(np.log1p, validate=True)),
                                    make_pipeline(FunctionTransformer(np.sqrt, validate=True))])
    model_features.activation_function = tf.nn.relu
    model_features.output_function = None
    model_features.Pooling = tf.layers.MaxPooling1D
    model_features.l2_regularization_scale = 10**np.random.uniform(-3,0)
    model_features.dropout_probability = np.random.uniform(0,1)
    model_features.pool_strides = ((2,2,2))
    number_layers = choice([2])
    dense_nodes = 2**np.random.randint(4,8,number_layers)
    dense_nodes = np.sort(dense_nodes)
    dense_nodes = np.flipud(dense_nodes)
    
    model_features.dense_nodes = dense_nodes

    model = CNN1D(model_features)

    return model, model_features 

### Search hyperparameters

In [ ]:
def save_features(model_features,
                  model_id,
                  hyperparameter_index):
    with open('./hyperparameter-search-results/' + model_id + '_' +
              str(hyperparameter_index), 'wb+') as f:
        pickle.dump(model_features,f)
    return None

skf = StratifiedKFold(n_splits=5, random_state=5)
testing_errors = []
all_kf_errors = []

for network_id in range(number_hyperparameters_to_search):
    print(network_id)
    model, model_features = make_model()
    save_features(model_features,
                  model_id,
                  network_id)
    
    k_folds_errors = []
    for train_index, test_index in skf.split(all_spectra, all_keys):
        # reset model on each iteration
        model = CNN1D(model_features)
        optimizer = tf.train.AdamOptimizer(model_features.learining_rate)

        costfunction_errors_tmp, earlystop_errors_tmp = train_earlystop(
                training_data=all_spectra[train_index],
                training_keys=all_keys_binarized[train_index],
                testing_data=all_spectra[test_index],
                testing_keys=all_keys_binarized[test_index],
                model=model,
                optimizer=optimizer,
                num_epochs=200,
                obj_cost=model.cross_entropy,
                earlystop_cost_fn=model.f1_error,
                earlystop_patience=10,
                not_learning_patience=10,
                not_learning_threshold=0.9,
                verbose=True,
                fit_batch_verbose=10,
                data_augmentation=model.default_data_augmentation)
        k_folds_errors.append(earlystop_errors_tmp)
        all_kf_errors.append(earlystop_errors_tmp)

    testing_errors.append(np.average(k_folds_errors))
    np.save('./final-models/final_test_errors_'+model_id, testing_errors)
    np.save('./final-models/final_kf_errors_'+model_id, all_kf_errors)
    # model.save_weights('./final-models/'+model_id+'_checkpoint_'+str(network_id))
    network_id += 1 

0
Epoch 10: CostFunc loss: 0.00 3.42, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.68 Early stopfctn: 3.68
Epoch 10: CostFunc loss: 0.00 3.49, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.72 Early stopfctn: 3.72
Epoch 10: CostFunc loss: 0.00 3.42, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.65 Early stopfctn: 3.65
Epoch 10: CostFunc loss: 0.00 3.41, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.54 Early stopfctn: 3.54
Epoch 10: CostFunc loss: 0.00 3.46, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.77 Early stopfctn: 3.77
1
Epoch 10: CostFunc loss: 0.00 0.39, EarlyStop loss: 0.00 0.07
Epoch 20: CostFunc loss: 0.00 0.23, EarlyStop loss: 0.00 0.01
Epoch 30: CostFunc loss: 0.00 0.19, EarlyStop loss: 0.00 0.01
Test error at early stop: Objectives fctn: 0.19 Early stopfctn: 0.19
Epoch 10: CostFunc loss: 0.00 0.55, EarlyStop loss: 0.00 0.07
Epoch 20: CostFunc loss:

Epoch 20: CostFunc loss: 0.00 0.80, EarlyStop loss: 0.00 0.17
Epoch 30: CostFunc loss: 0.00 0.71, EarlyStop loss: 0.00 0.17
Epoch 40: CostFunc loss: 0.00 0.61, EarlyStop loss: 0.00 0.10
Test error at early stop: Objectives fctn: 0.61 Early stopfctn: 0.61
Epoch 10: CostFunc loss: 0.00 0.83, EarlyStop loss: 0.00 0.19
Epoch 20: CostFunc loss: 0.00 0.70, EarlyStop loss: 0.00 0.11
Epoch 30: CostFunc loss: 0.00 0.62, EarlyStop loss: 0.00 0.12
Epoch 40: CostFunc loss: 0.00 0.56, EarlyStop loss: 0.00 0.10
Test error at early stop: Objectives fctn: 0.54 Early stopfctn: 0.54
Epoch 10: CostFunc loss: 0.00 0.89, EarlyStop loss: 0.00 0.23
Epoch 20: CostFunc loss: 0.00 0.68, EarlyStop loss: 0.00 0.10
Epoch 30: CostFunc loss: 0.00 0.61, EarlyStop loss: 0.00 0.12
Test error at early stop: Objectives fctn: 0.62 Early stopfctn: 0.62
Epoch 10: CostFunc loss: 0.00 0.93, EarlyStop loss: 0.00 0.21
Epoch 20: CostFunc loss: 0.00 0.71, EarlyStop loss: 0.00 0.15
Epoch 30: CostFunc loss: 0.00 0.58, EarlyStop los

Epoch 10: CostFunc loss: 0.00 3.75, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 5.67 Early stopfctn: 5.67
Epoch 10: CostFunc loss: 0.00 4.62, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 9.56 Early stopfctn: 9.56
Epoch 10: CostFunc loss: 0.00 5.07, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 11.61 Early stopfctn: 11.61
19
Epoch 10: CostFunc loss: 0.00 3.41, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.47 Early stopfctn: 3.47
Epoch 10: CostFunc loss: 0.00 3.40, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.41 Early stopfctn: 3.41
Epoch 10: CostFunc loss: 0.00 1.08, EarlyStop loss: 0.00 0.23
Epoch 20: CostFunc loss: 0.00 0.95, EarlyStop loss: 0.00 0.17
Epoch 30: CostFunc loss: 0.00 0.77, EarlyStop loss: 0.00 0.13
Epoch 40: CostFunc loss: 0.00 0.73, EarlyStop loss: 0.00 0.11
Epoch 50: CostFunc loss: 0.00 0.81, EarlyStop loss: 0.00 0.15
Test error at early stop: Obje

Epoch 50: CostFunc loss: 0.00 0.99, EarlyStop loss: 0.00 0.32
Epoch 60: CostFunc loss: 0.00 1.05, EarlyStop loss: 0.00 0.31
Epoch 70: CostFunc loss: 0.00 0.93, EarlyStop loss: 0.00 0.31
Test error at early stop: Objectives fctn: 0.90 Early stopfctn: 0.90
30
Epoch 10: CostFunc loss: 0.00 3.55, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.58 Early stopfctn: 3.58
Epoch 10: CostFunc loss: 0.00 3.56, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.59 Early stopfctn: 3.59
Epoch 10: CostFunc loss: 0.00 3.62, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.64 Early stopfctn: 3.64
Epoch 10: CostFunc loss: 0.00 3.62, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.65 Early stopfctn: 3.65
Epoch 10: CostFunc loss: 0.00 3.56, EarlyStop loss: 0.00 0.96
Test error at early stop: Objectives fctn: 3.59 Early stopfctn: 3.59
31
Epoch 10: CostFunc loss: 0.00 3.78, EarlyStop loss: 0.00 0.97
Test error at early st

Epoch 10: CostFunc loss: 0.00 3.63, EarlyStop loss: 0.00 0.94
Test error at early stop: Objectives fctn: 4.02 Early stopfctn: 4.02
Epoch 10: CostFunc loss: 0.00 4.78, EarlyStop loss: 0.00 1.00
Test error at early stop: Objectives fctn: 6.36 Early stopfctn: 6.36
Epoch 10: CostFunc loss: 0.00 3.86, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 4.66 Early stopfctn: 4.66
Epoch 10: CostFunc loss: 0.00 3.72, EarlyStop loss: 0.00 0.96
Test error at early stop: Objectives fctn: 4.29 Early stopfctn: 4.29
41
Epoch 10: CostFunc loss: 0.00 0.36, EarlyStop loss: 0.00 0.05
Epoch 20: CostFunc loss: 0.00 0.24, EarlyStop loss: 0.00 0.03
Test error at early stop: Objectives fctn: 0.23 Early stopfctn: 0.23
Epoch 10: CostFunc loss: 0.00 0.45, EarlyStop loss: 0.00 0.05
Epoch 20: CostFunc loss: 0.00 0.34, EarlyStop loss: 0.00 0.05
Epoch 30: CostFunc loss: 0.00 0.22, EarlyStop loss: 0.00 0.02
Epoch 40: CostFunc loss: 0.00 0.18, EarlyStop loss: 0.00 0.01
Test error at early stop: Object

Test error at early stop: Objectives fctn: 1.38 Early stopfctn: 1.38
48
Epoch 10: CostFunc loss: 0.00 3.07, EarlyStop loss: 0.00 0.87
Epoch 20: CostFunc loss: 0.00 2.70, EarlyStop loss: 0.00 0.82
Epoch 30: CostFunc loss: 0.00 2.24, EarlyStop loss: 0.00 0.61
Epoch 40: CostFunc loss: 0.00 1.77, EarlyStop loss: 0.00 0.42
Epoch 50: CostFunc loss: 0.00 1.47, EarlyStop loss: 0.00 0.33
Epoch 60: CostFunc loss: 0.00 1.10, EarlyStop loss: 0.00 0.28
Epoch 70: CostFunc loss: 0.00 0.80, EarlyStop loss: 0.00 0.15
Epoch 80: CostFunc loss: 0.00 0.65, EarlyStop loss: 0.00 0.12
Epoch 90: CostFunc loss: 0.00 0.55, EarlyStop loss: 0.00 0.07
Epoch 100: CostFunc loss: 0.00 0.45, EarlyStop loss: 0.00 0.04
Epoch 110: CostFunc loss: 0.00 0.43, EarlyStop loss: 0.00 0.04
Test error at early stop: Objectives fctn: 0.45 Early stopfctn: 0.45
Epoch 10: CostFunc loss: 0.00 1.68, EarlyStop loss: 0.00 0.31
Epoch 20: CostFunc loss: 0.00 0.95, EarlyStop loss: 0.00 0.12
Epoch 30: CostFunc loss: 0.00 0.69, EarlyStop loss: